In [1]:
!pip uninstall -y scikit-learn imblearn xgboost
!pip install scikit-learn>=1.5.0 imblearn>=0.12.0 xgboost>=2.0.0

import sklearn
import imblearn
import xgboost

print(f"scikit-learn version: {sklearn.__version__}")
print(f"imblearn version: {imblearn.__version__}")
print(f"xgboost version: {xgboost.__version__}")

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: xgboost 2.0.3
Uninstalling xgboost-2.0.3:
  Successfully uninstalled xgboost-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
scikit-learn version: 1.6.1
imblearn version: 0.13.0
xgboost version: 3.0.2


In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, make_scorer, precision_recall_fscore_support,
                             roc_auc_score, roc_curve, classification_report)
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np
import io
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
import itertools
import tensorflow as tf

import cv2
from tensorflow.keras.applications import InceptionResNetV2, ResNet152V2, DenseNet169
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

dataset_save_path = "/kaggle/working/"
model_data_load_save_path = "/kaggle/working/"
image_load_path = "/kaggle/input/cropped-external-egg-image"

2025-06-06 05:31:29.333814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749187889.598056      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749187889.676213      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
# For Grade
df=pd.read_csv("/kaggle/input/new-dataset/New-Egg-dataset-with-Train-Test.csv")
df = df[["Egg No", 'image_name', "Egg weight (g)", "Shape index", 'Egg_grade']]

#MERGE CODE
def merge_class(cls):
    if(cls == "A" or cls == "AA"):
      return "High"
    else:
      return "Low"
df['New_Egg_grade'] = df['Egg_grade'].apply(
    lambda cls: merge_class(cls)
)

df = df.dropna(subset=["Egg No", 'image_name', "Egg weight (g)", "Shape index", 'Egg_grade'])
print(df['New_Egg_grade'].value_counts())

New_Egg_grade
Low     108
High     78
Name: count, dtype: int64


In [5]:
#For Grade

model = ResNet152V2(weights='imagenet', include_top=False, pooling='avg')

def make_trainable_feature(train_df):
    Train_feature_list = []
    Train_weight_list = []
    Train_shape_index_list = []
    egg_info = train_df[['image_name', 'Egg weight (g)', 'Shape index']].set_index('image_name')[['Egg weight (g)', 'Shape index']].to_dict('index')
    
    for filename in train_df['image_name'].values:
        image_path = os.path.join(image_load_path, filename)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array)
        Train_feature_list.append(features.flatten())
        egg_data = egg_info.get(filename, {})
        Train_weight_list.append(egg_data.get('Egg weight (g)'))
        Train_shape_index_list.append(egg_data.get('Shape index'))
    
    Train_features_df = pd.DataFrame(Train_feature_list, columns=[f'feature_{i}' for i in range(Train_feature_list[0].shape[0])])
    weight_df = pd.DataFrame(Train_weight_list, columns=["weight"])
    shape_df = pd.DataFrame(Train_shape_index_list, columns=["shape_index"])
    weight_shape_df = pd.concat([weight_df, shape_df], axis=1)
    Train_features_df = pd.concat([weight_shape_df, Train_features_df], axis=1)
    Train_features_df['New_Egg_grade'] = train_df['New_Egg_grade']
    Train_features_df = Train_features_df.dropna(subset=["New_Egg_grade"])
    
    return Train_features_df

features_df = make_trainable_feature(df)

2025-06-06 05:31:45.108864: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━

In [6]:
features_df.to_csv('Org_Merged_ResNet152V2_Grade_2C_Egg_Image_Feature_Dataset.csv', index=False)

In [7]:
#For Grade
label_encoder = LabelEncoder()
scaler = StandardScaler()
pca = PCA(n_components=0.99)
smote = SMOTE(random_state=42)

feature_cols = [col for col in features_df.columns if col.startswith('feature_')]
feature_cols.append("weight")
feature_cols.append("shape_index")
Train_features_df = features_df[feature_cols]
y = label_encoder.fit_transform(features_df['New_Egg_grade'])

X_train_scaled = scaler.fit_transform(Train_features_df)
oversampled_data, y_smote = smote.fit_resample(X_train_scaled, y)
scaled_cols = [f'Scaled{i+1}' for i in range(oversampled_data.shape[1])]
# scaled_cols = [f'Scaled{i+1}' for i in range(X_train_scaled.shape[1])]
scaled_df = pd.DataFrame(oversampled_data, columns=scaled_cols)

X_train_pca = pca.fit_transform(oversampled_data)
pca_cols = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]
pca_df = pd.DataFrame(X_train_pca, columns=pca_cols)

scaled_pca_df = pd.concat([pca_df, scaled_df], axis=1)
target = label_encoder.inverse_transform(y_smote)
scaled_pca_df["New_Egg_grade"] = target

shuffled_new_df = scaled_pca_df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_new_df.to_csv('Merged_ResNet152V2_Grade_2C_Egg_Image_Feature_Dataset.csv', index=False)

In [8]:
# For Freshness
df=pd.read_csv("/kaggle/input/new-dataset/New-Egg-dataset-with-Train-Test.csv")
df = df[["Egg No", 'image_name', "Egg weight (g)", "Shape index", 'Egg_freshness']]

#MERGE CODE
def merge_class(cls):
    if(cls == "Fresh" or cls == "ModeratelyFresh"):
      return "New_Fresh"
    else:
      return "Old"
df['New_Egg_freshness'] = df['Egg_freshness'].apply(
    lambda cls: merge_class(cls)
)

df = df.dropna(subset=["Egg No", 'image_name', "Egg weight (g)", "Shape index", 'Egg_freshness'])
print(df['New_Egg_freshness'].value_counts())

New_Egg_freshness
Old          96
New_Fresh    90
Name: count, dtype: int64


In [9]:
#For Freshness
model = ResNet152V2(weights='imagenet', include_top=False, pooling='avg')

def make_trainable_feature(train_df):
    Train_feature_list = []
    Train_weight_list = []
    Train_shape_index_list = []
    egg_info = train_df[['image_name', 'Egg weight (g)', 'Shape index']].set_index('image_name')[['Egg weight (g)', 'Shape index']].to_dict('index')
    
    for filename in train_df['image_name'].values:
        image_path = os.path.join(image_load_path, filename)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array)
        Train_feature_list.append(features.flatten())
        egg_data = egg_info.get(filename, {})
        Train_weight_list.append(egg_data.get('Egg weight (g)'))
        Train_shape_index_list.append(egg_data.get('Shape index'))
    
    Train_features_df = pd.DataFrame(Train_feature_list, columns=[f'feature_{i}' for i in range(Train_feature_list[0].shape[0])])
    weight_df = pd.DataFrame(Train_weight_list, columns=["weight"])
    shape_df = pd.DataFrame(Train_shape_index_list, columns=["shape_index"])
    weight_shape_df = pd.concat([weight_df, shape_df], axis=1)
    Train_features_df = pd.concat([weight_shape_df, Train_features_df], axis=1)
    Train_features_df['New_Egg_freshness'] = train_df['New_Egg_freshness']
    Train_features_df = Train_features_df.dropna(subset=["New_Egg_freshness"])
    
    return Train_features_df

features_df = make_trainable_feature(df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [10]:
features_df.to_csv('Org_Merged_ResNet152V2_Freshness_2C_Egg_Image_Feature_Dataset.csv', index=False)

In [11]:
#For Freshness
label_encoder = LabelEncoder()
scaler = StandardScaler()
pca = PCA(n_components=0.99)
smote = SMOTE(random_state=42)

feature_cols = [col for col in features_df.columns if col.startswith('feature_')]
feature_cols.append("weight")
feature_cols.append("shape_index")
Train_features_df = features_df[feature_cols]
y = label_encoder.fit_transform(features_df['New_Egg_freshness'])

X_train_scaled = scaler.fit_transform(Train_features_df)
oversampled_data, y_smote = smote.fit_resample(X_train_scaled, y)
scaled_cols = [f'Scaled{i+1}' for i in range(oversampled_data.shape[1])]
# scaled_cols = [f'Scaled{i+1}' for i in range(X_train_scaled.shape[1])]
scaled_df = pd.DataFrame(oversampled_data, columns=scaled_cols)

X_train_pca = pca.fit_transform(oversampled_data)
pca_cols = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]
pca_df = pd.DataFrame(X_train_pca, columns=pca_cols)

scaled_pca_df = pd.concat([pca_df, scaled_df], axis=1)
target = label_encoder.inverse_transform(y_smote)
scaled_pca_df["New_Egg_freshness"] = target

shuffled_new_df = scaled_pca_df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_new_df.to_csv('Merged_ResNet152V2_Freshness_2C_Egg_Image_Feature_Dataset.csv', index=False)